In [1]:
pip install pygame

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import pygame
import sys

# Constantes
BOARD_SIZE = 5
CELL_SIZE = 100
WINDOW_SIZE = BOARD_SIZE * CELL_SIZE
BUTTON_HEIGHT = 50
COLORS = {
    'WHITE': (255, 255, 255),
    'BLACK': (0, 0, 0),
    'RED': (255, 0, 0),
    'BLUE': (0, 0, 255),
    'GREEN': (0, 255, 0),
    'YELLOW': (255, 255, 0),  # Couleur pour les cases valides
    'GRAY': (200, 200, 200),  # Couleur pour les boutons
}
PLAYER_COLORS = {'X': 'BLUE', 'O': 'GREEN'}

class BobailGame:
    def __init__(self):
        self.reset()

    def reset(self):
        """Réinitialise le jeu."""
        # Initialisation du plateau de jeu 5x5
        self.board = [[' ' for _ in range(BOARD_SIZE)] for _ in range(BOARD_SIZE)]
        
        # Position initiale du BOBAIL (rouge) au centre
        self.board[2][2] = 'R'
        
        # Positions initiales des pions des joueurs
        # Joueur 1 (Pions 'X')
        for i in range(BOARD_SIZE):
            self.board[0][i] = 'X'
        
        # Joueur 2 (Pions 'O')
        for i in range(BOARD_SIZE):
            self.board[BOARD_SIZE - 1][i] = 'O'
        
        # Tour du joueur 1 (X) par défaut
        self.current_player = 'X'
        
        # État du jeu (en cours, gagné, etc.)
        self.game_over = False
        self.winner = None

    def is_valid_move(self, start, end):
        """Vérifie si un déplacement est valide."""
        x1, y1 = start
        x2, y2 = end
        
        # Vérifie si les coordonnées sont dans les limites du plateau
        if not (0 <= x1 < BOARD_SIZE and 0 <= y1 < BOARD_SIZE and 0 <= x2 < BOARD_SIZE and 0 <= y2 < BOARD_SIZE):
            return False
        
        # Vérifie si la case de départ contient un pion du joueur actuel ou le BOBAIL
        if self.board[x1][y1] != self.current_player and self.board[x1][y1] != 'R':
            return False
        
        # Vérifie si la case d'arrivée est vide
        if self.board[x2][y2] != ' ':
            return False
        
        # Vérifie le type de déplacement (horizontal, vertical, diagonal)
        dx = x2 - x1
        dy = y2 - y1
        if dx != 0 and dy != 0 and abs(dx) != abs(dy):
            return False  # Pas en diagonale
        
        # Règles spécifiques pour les pions et le BOBAIL
        if self.board[x1][y1] == 'R':  # Déplacement du BOBAIL
            # Le BOBAIL ne peut se déplacer que d'une case
            if abs(dx) > 1 or abs(dy) > 1:
                return False
        else:  # Déplacement d'un pion
            # Les pions doivent aller jusqu'au bout de leur rangée
            step_x = 1 if dx > 0 else -1 if dx < 0 else 0
            step_y = 1 if dy > 0 else -1 if dy < 0 else 0
            x, y = x1 + step_x, y1 + step_y
            while x != x2 or y != y2:
                if self.board[x][y] != ' ':
                    return False
                x += step_x
                y += step_y
        
        return True

    def get_valid_moves(self, start):
        """Retourne les cases valides pour un déplacement à partir d'une position donnée."""
        valid_moves = []
        x1, y1 = start
        if self.board[x1][y1] == 'R':  # Déplacements du BOBAIL
            directions = [
                (1, 0), (-1, 0),  # Horizontal
                (0, 1), (0, -1),  # Vertical
                (1, 1), (1, -1), (-1, 1), (-1, -1)  # Diagonales
            ]
            for dx, dy in directions:
                x, y = x1 + dx, y1 + dy
                if 0 <= x < BOARD_SIZE and 0 <= y < BOARD_SIZE and self.board[x][y] == ' ':
                    valid_moves.append((x, y))
        else:  # Déplacements des pions
            directions = [
                (1, 0), (-1, 0),  # Horizontal
                (0, 1), (0, -1),  # Vertical
                (1, 1), (1, -1), (-1, 1), (-1, -1)  # Diagonales
            ]
            for dx, dy in directions:
                x, y = x1 + dx, y1 + dy
                while 0 <= x < BOARD_SIZE and 0 <= y < BOARD_SIZE:
                    if self.board[x][y] != ' ':
                        break  # Obstacle rencontré
                    valid_moves.append((x, y))
                    x += dx
                    y += dy
        return valid_moves

    def move_bobail(self, end):
        """Déplace le BOBAIL."""
        x, y = end
        if self.board[x][y] == ' ':
            # Trouver la position actuelle du BOBAIL
            for i in range(BOARD_SIZE):
                for j in range(BOARD_SIZE):
                    if self.board[i][j] == 'R':
                        self.board[i][j] = ' '
                        break
            self.board[x][y] = 'R'
            return True
        return False

    def move_pawn(self, start, end):
        """Déplace un pion."""
        x1, y1 = start
        x2, y2 = end
        if self.is_valid_move(start, end):
            self.board[x2][y2] = self.board[x1][y1]
            self.board[x1][y1] = ' '
            return True
        return False

    def check_win(self):
        """Vérifie si un joueur a gagné."""
        # Vérifie si le BOBAIL est sur un trou de départ
        for i in range(BOARD_SIZE):
            if self.board[0][i] == 'R':
                self.game_over = True
                self.winner = 'X'
                return True
            if self.board[BOARD_SIZE - 1][i] == 'R':
                self.game_over = True
                self.winner = 'O'
                return True
        
        # Vérifie si le BOBAIL est bloqué
        # (À implémenter selon les règles exactes)
        
        return False

    def play_turn(self, bobail_move=None, pawn_move=None):
        """Joue un tour."""
        if self.game_over:
            return
        
        # Déplacement du BOBAIL
        if bobail_move is not None:
            if not self.move_bobail(bobail_move):
                raise ValueError("Déplacement du BOBAIL invalide.")
        
        # Déplacement d'un pion
        if pawn_move is not None:
            start, end = pawn_move
            if not self.move_pawn(start, end):
                raise ValueError("Déplacement du pion invalide.")
        
        # Vérifie si le jeu est gagné
        if self.check_win():
            self.game_over = True
        else:
            # Change de joueur
            self.current_player = 'O' if self.current_player == 'X' else 'X'

# Interface graphique avec Pygame
class BobailGUI:
    def __init__(self, game):
        self.game = game
        pygame.init()
        self.screen = pygame.display.set_mode((WINDOW_SIZE, WINDOW_SIZE + BUTTON_HEIGHT))
        pygame.display.set_caption("BOBAIL")
        self.font = pygame.font.Font(None, 36)
        self.selected_pawn = None
        self.valid_moves = []
        self.animating = False
        self.animation_start_pos = None
        self.animation_end_pos = None
        self.animation_progress = 0

    def draw_board(self):
        """Dessine le plateau de jeu."""
        self.screen.fill(COLORS['WHITE'])
        for x in range(BOARD_SIZE):
            for y in range(BOARD_SIZE):
                rect = pygame.Rect(y * CELL_SIZE, x * CELL_SIZE, CELL_SIZE, CELL_SIZE)
                pygame.draw.rect(self.screen, COLORS['BLACK'], rect, 1)
                if (x, y) in self.valid_moves:
                    pygame.draw.rect(self.screen, COLORS['YELLOW'], rect)  # Cases valides
                if self.game.board[x][y] == 'R':
                    pygame.draw.circle(self.screen, COLORS['RED'], rect.center, CELL_SIZE // 3)
                elif self.game.board[x][y] == 'X':
                    pygame.draw.circle(self.screen, COLORS['BLUE'], rect.center, CELL_SIZE // 3)
                elif self.game.board[x][y] == 'O':
                    pygame.draw.circle(self.screen, COLORS['GREEN'], rect.center, CELL_SIZE // 3)
        
        # Affiche à qui de jouer
        turn_text = self.font.render(f"Tour du joueur {self.game.current_player}", True, COLORS['BLACK'])
        self.screen.blit(turn_text, (10, WINDOW_SIZE + 10))

        # Boutons
        self.draw_button("Redémarrer", WINDOW_SIZE // 4 - 50, WINDOW_SIZE + 10, 100, BUTTON_HEIGHT - 20, COLORS['GRAY'])
        self.draw_button("Quitter", 3 * WINDOW_SIZE // 4 - 50, WINDOW_SIZE + 10, 100, BUTTON_HEIGHT - 20, COLORS['GRAY'])

    def draw_button(self, text, x, y, width, height, color):
        """Dessine un bouton."""
        pygame.draw.rect(self.screen, color, (x, y, width, height))
        text_surface = self.font.render(text, True, COLORS['BLACK'])
        text_rect = text_surface.get_rect(center=(x + width // 2, y + height // 2))
        self.screen.blit(text_surface, text_rect)

    def get_cell_from_mouse(self, pos):
        """Retourne la cellule correspondante à la position de la souris."""
        x, y = pos
        row = y // CELL_SIZE
        col = x // CELL_SIZE
        return (row, col)

    def handle_click(self, pos):
        """Gère les clics de souris."""
        x, y = pos
        if y < WINDOW_SIZE:  # Clic sur le plateau
            cell = self.get_cell_from_mouse(pos)
            if self.selected_pawn is None:
                # Sélection du BOBAIL ou d'un pion
                if self.game.board[cell[0]][cell[1]] == 'R' or self.game.board[cell[0]][cell[1]] == self.game.current_player:
                    self.selected_pawn = cell
                    self.valid_moves = self.game.get_valid_moves(cell)
            else:
                # Déplacement du BOBAIL ou du pion
                if cell in self.valid_moves:
                    self.animation_start_pos = self.selected_pawn
                    self.animation_end_pos = cell
                    self.animating = True
                    self.animation_progress = 0
        else:  # Clic sur les boutons
            if WINDOW_SIZE // 4 - 50 <= x <= WINDOW_SIZE // 4 + 50:
                self.game.reset()  # Redémarrer
            elif 3 * WINDOW_SIZE // 4 - 50 <= x <= 3 * WINDOW_SIZE // 4 + 50:
                pygame.quit()  # Quitter
                sys.exit()

    def run(self):
        """Boucle principale du jeu."""
        clock = pygame.time.Clock()
        while True:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()
                
                if event.type == pygame.MOUSEBUTTONDOWN:
                    self.handle_click(event.pos)
            
            # Animation
            if self.animating:
                self.animation_progress += 0.05
                if self.animation_progress >= 1:
                    self.animating = False
                    try:
                        if self.game.board[self.animation_start_pos[0]][self.animation_start_pos[1]] == 'R':
                            self.game.play_turn(bobail_move=self.animation_end_pos)
                        else:
                            self.game.play_turn(pawn_move=(self.animation_start_pos, self.animation_end_pos))
                        self.selected_pawn = None
                        self.valid_moves = []
                    except ValueError as e:
                        print(e)
                        self.selected_pawn = None
                        self.valid_moves = []
            
            self.draw_board()
            if self.animating:
                # Dessine l'animation
                start_x = self.animation_start_pos[1] * CELL_SIZE + CELL_SIZE // 2
                start_y = self.animation_start_pos[0] * CELL_SIZE + CELL_SIZE // 2
                end_x = self.animation_end_pos[1] * CELL_SIZE + CELL_SIZE // 2
                end_y = self.animation_end_pos[0] * CELL_SIZE + CELL_SIZE // 2
                current_x = start_x + (end_x - start_x) * self.animation_progress
                current_y = start_y + (end_y - start_y) * self.animation_progress
                if self.game.board[self.animation_start_pos[0]][self.animation_start_pos[1]] == 'R':
                    pygame.draw.circle(self.screen, COLORS['RED'], (int(current_x), int(current_y)), CELL_SIZE // 3)
                else:
                    color = COLORS['BLUE'] if self.game.current_player == 'X' else COLORS['GREEN']
                    pygame.draw.circle(self.screen, color, (int(current_x), int(current_y)), CELL_SIZE // 3)
            
            if self.game.game_over:
                text = self.font.render(f"Le joueur {self.game.winner} a gagné!", True, COLORS['BLACK'])
                self.screen.blit(text, (WINDOW_SIZE // 4, WINDOW_SIZE // 2))
            
            pygame.display.flip()
            clock.tick(30)  # Limite à 30 FPS

# Lancement du jeu
if __name__ == "__main__":
    game = BobailGame()
    gui = BobailGUI(game)
    gui.run()

SystemExit: 